## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

## Input data directory
data_dir = "OrfPathHealth"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
## Dir Loader
documents = PyPDFDirectoryLoader(inputdirectory).load()
## File Loader
# documents = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf").load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))


## Create a dataframe of all the chunks

In [3]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

## Extract Concepts

In [4]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2ConceptsList
from helpers.df_helpers import concepts2Df

In [9]:
concepts_list = df2ConceptsList(df[10:12])

In [26]:
dfne = concepts2Df(concepts_list)
dfne.head()

### Write CSV to an output directory

Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of names entities

        df = dataframe of chunks

In [28]:
if not os.path.exists(outputdirectory):
   os.makedirs(outputdirectory)
   
dfne.to_csv(outputdirectory/"concepts.csv", sep="|", index=False)
df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)

## Named Entities from Concepts

**Not using this right now**

Extracting named entities our of concepts. 


In [3]:
from transformers import pipeline

ner = pipeline("token-classification", model="2rtl3/mn-xlm-roberta-base-named-entity", aggregation_strategy="simple")
# ner = pipeline("token-classification", model="dslim/bert-large-NER", aggregation_strategy="simple")

def row2NamedEntities(row):
    ner_results = ner(row['entity'])
    metadata = {'chunk_id': row['chunk_id'], 'type': 'entity'}
    entities = []
    for result in ner_results:
        entities = entities + [{'entity': result['word'], 'catetory': result['entity_group'], **metadata}]
        
    return entities



def dfText2DfNE(dataframe: pd.DataFrame):
    ## Takes a dataframe from the parsed data and returns dataframe with named entities. 
    ## The input dataframe must have a entity and a chunk_id column. 

    ## 1. Calculate named entities for each row of the dataframe. 
    results = dataframe.apply(row2NamedEntities, axis=1).reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities. 
    entities_list = np.concatenate(results).ravel().tolist()

    ## Remove all NaN entities
    entities_dataframe = pd.DataFrame(entities_list).replace(' ', np.nan)
    entities_dataframe = entities_dataframe.dropna(subset=['entity'])

    ## Count the number of occurances per chunk id
    # entities_dataframe = entities_dataframe.groupby(['entity', 'category', 'chunk_id']).size().reset_index(name='count')

    return entities_dataframe

In [5]:

dataframe_dir = 'OrfPathHealth'
df_concepts = pd.read_csv(f"./data/output/{dataframe_dir}/concepts.csv", sep="|")

dfc_split = dfText2DfNE(df_concepts)
dfc_split